In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import voila

# Store profile entries
profiles = []

# Output area for profile table
profile_out = widgets.Output()

layout = widgets.Layout(width='400px')  # Wider total box if needed
style = {'description_width': '150px'}  # Wider label

# Input widgets
account_name = widgets.Text(description='Brand Name:', layout=layout, style=style)
fb_name = widgets.Text(description='Facebook Handle:', layout=layout, style=style)
inst_name = widgets.Text(description='Instagram Handle:', layout=layout, style=style)
tt_name = widgets.Text(description='TikTok Handle:', layout=layout, style=style)

# Button to add profile
add_button = widgets.Button(description='Add Profile', button_style='primary')
clear_button = widgets.Button(description='Clear All', button_style='warning')

# Display profiles
def display_profiles():
    profile_out.clear_output()
    if profiles:
        df = pd.DataFrame(profiles)
        with profile_out:
            display(df)

# Button click logic
def on_add_clicked(_):
    profiles.append({
        'account_name': account_name.value.strip(),
        'fb_profile_name': fb_name.value.strip(),
        'inst_profile_name': inst_name.value.strip(),
        'tt_profile_name': tt_name.value.strip()
    })
    account_name.value = fb_name.value = inst_name.value = tt_name.value = ''
    display_profiles()

def on_clear_clicked(_):
    profiles.clear()
    profile_out.clear_output()

add_button.on_click(on_add_clicked)
clear_button.on_click(on_clear_clicked)

# Layout for inputs
input_box = widgets.VBox([
    account_name, fb_name, inst_name, tt_name,
    widgets.HBox([add_button, clear_button]),
    profile_out
])

display(widgets.HTML("<h2>📋 Enter Social Media Profiles</h2>"))
display(input_box)

ModuleNotFoundError: No module named 'voila'

In [1]:
def fetch_data():
           
    # ## Preparation
    
    # In[36]:
    
    
    import pandas as pd
    from apify_client import ApifyClient
    import json
    pd.options.display.max_columns = None
    from datetime import datetime, timedelta, date
    from pandas import json_normalize
    import numpy as np
    import math
    from sqlalchemy.exc import DatabaseError
    from openai import OpenAI, AsyncOpenAI
    import os
    from tqdm import tqdm
    tqdm.pandas()
    from typing import Dict, List
    import tiktoken
    import nest_asyncio, asyncio
    import dateparser
    import plotly
    import matplotlib.pyplot as plt
    import matplotlib
    import matplotlib.ticker as ticker
    from matplotlib.backends.backend_pdf import PdfPages
    from PIL import Image
    from docx import Document
    from docx.shared import Inches
    from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
    from docx.shared import Inches, Pt
    from docx.oxml.ns import qn
    from docx.oxml import OxmlElement
    from docx.enum.table import WD_ALIGN_VERTICAL
    from docx.shared import Pt, RGBColor
    from io import BytesIO
    from docx2pdf import convert
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    from IPython.display import FileLink
    print('Run Started')
    import pathlib
    
    
    # ### API Keys
    
    # In[27]:
    
    
    apify = ApifyClient("apify_api_KP7TYMkOEwszNWMuGjWqm8cUeAxoFs2pvmJF")
    nest_asyncio.apply()
    
    
    # ### Profile Names
    
    # In[5]:
    
    
    # Create Lookup Table
    lookup = []
    for profile in profiles:
        for platform, key in {
            'Facebook': 'fb_profile_name',
            'Instagram': 'inst_profile_name',
            'Tiktok': 'tt_profile_name',
        }.items():
            lookup.append({
                'platform': platform,
                'user_name': profile[key],
                'account_name': profile['account_name']
            })
    
    lookup_df = pd.DataFrame(lookup)
    lookup_df['user_name'] = lookup_df['user_name'].str.lower()
    
    
    # In[6]:
    
    
    # Collect every non-empty Facebook profile name
    fb_profile_names = [
        p["fb_profile_name"]
        for p in profiles
        if p.get("fb_profile_name")        # skips None or missing/empty entries
    ]
    
    
    # In[7]:
    
    
    # Collect every non-empty Instagram profile name
    inst_profile_names = [
        p["inst_profile_name"]
        for p in profiles
        if p.get("inst_profile_name")        # skips None or missing/empty entries
    ]
    
    
    # In[8]:
    
    
    # Collect every non-empty TikTok profile name
    tt_profile_names = [
        p["tt_profile_name"]
        for p in profiles
        if p.get("tt_profile_name")        # skips None or missing/empty entries
    ]
    
    
    # In[9]:
    
    
    # Collect every non-empty YouTube profile name
    yt_profile_names = [
        p["yt_profile_name"]
        for p in profiles
        if p.get("yt_profile_name")        # skips None or missing/empty entries
    ]
    
    
    # In[10]:
    
    
    # Collect every non-empty X profile name
    x_profile_names = [
        p["x_profile_name"]
        for p in profiles
        if p.get("x_profile_name")        # skips None or missing/empty entries
    ]
    
    
    # ### Global Parameters
    
    # In[11]:
    
    
    max_results = 500
    past_days_posts = 14 #Change to 3 after initial run
    #past_days_comments = 3 #Change to 3 after initial run
    
    start_date_posts = (datetime.now() + timedelta(-past_days_posts)).strftime("%Y-%m-%d")
    #start_date_comments = (datetime.now() + timedelta(-past_days_comments)).strftime("%Y-%m-%d")
    
    
    # In[38]:
    
    
    # Set smoother font globally
    matplotlib.rcParams['font.family'] = 'DejaVu Sans'
    
    
    # ## Posts
    
    # ### Facebook
    
    # In[12]:
    
    
    # Facebook Parameters
    
    fb_max_results = max_results
    fb_start_date_posts = start_date_posts
    
    
    # In[13]:
    
    if fb_profile_names:
        starturls = [f"https://www.facebook.com/{name}/" for name in fb_profile_names]
        i = 0
        data = {}
        
        for url in starturls:
        
            run_input = {
        
                "page_url": url,
                "start_date": fb_start_date_posts,
                "max_posts": fb_max_results
            }
        
            # Run the Actor and wait for it to finish
            run = apify.actor("danek/facebook-pages-posts-ppr").call(run_input=run_input)
        
        
            for item in apify.dataset(run["defaultDatasetId"]).iterate_items():
                data[i] = item
                i = i + 1
        
        
        
        # In[14]:
        
        
        posts = pd.DataFrame(data).transpose()
        posts['facebookUrl'] = posts['author'].apply(lambda x: x.get('url') if isinstance(x, dict) else None)
        posts['pageName'] = posts['facebookUrl'].str.extract(r'facebook\.com/([^/?#]+)')
        posts['time'] = pd.to_datetime(posts['timestamp'], unit='s', utc=True)
        posts['time'] = posts['time'].dt.tz_convert('Africa/Johannesburg') 
        posts['likes'] = posts['reactions'].apply(lambda x: x.get('likes') if isinstance(x, dict) else None)
        posts['isVideo'] = np.where(posts['video'].isna(), 'Text', 'Video')
        posts['viewsCount'] = np.nan
        
        
        # In[15]:
        
        
        fb_posts = posts[['facebookUrl', 'post_id', 'pageName', 'url', 'time', 'message', 'likes', 'comments_count', 'reshare_count', 'viewsCount', 'isVideo']]
        
        fb_posts = fb_posts.rename(columns={
        'facebookUrl': 'profile_url',
        'post_id': 'id',
        'pageName': 'user_name',
        'url': 'url',
        'time': 'date',
        'message': 'text',
        'likes': 'likes',
        'comments_count': 'comments',
        'reshare_count': 'shares',
        'viewsCount': 'views',
        'isVideo': 'type'    })
        
    # ### Instagram
    
    # In[16]:
    
    
    # Instagram Parameters
    
    inst_max_results = max_results
    inst_start_date_posts = start_date_posts
    
    
    # In[17]:
    
    if inst_profile_names:
        starturls = [f"https://www.instagram.com/{username}" for username in inst_profile_names]
        
        run_input = {
        
            "directUrls": starturls,
            "resultsType": "posts",
            "resultsLimit": inst_max_results,
            "onlyPostsNewerThan": inst_start_date_posts
        }
        
        # Run the Actor and wait for it to finish
        run = apify.actor("apify/instagram-scraper").call(run_input=run_input)
        
        i = 0
        data = {}
        for item in apify.dataset(run["defaultDatasetId"]).iterate_items():
            data[i] = item
            i = i +1
        
        posts = pd.DataFrame(data).transpose()
        inst_posts = posts[['inputUrl', 'id', 'ownerUsername', 'url', 'timestamp', 'caption', 'likesCount', 'commentsCount', 'videoPlayCount', 'type', 'videoDuration']]
        
        
        # In[18]:
        
        
        # Get true username
        
        inst_posts['ownerUsername'] = inst_posts['inputUrl'].str.extract(r'instagram\.com/([^/?#]+)')

        inst_posts = inst_posts.rename(columns={
        'inputUrl': 'profile_url',
        'id': 'id',
        'ownerUsername': 'user_name',
        'url': 'url',
        'timestamp': 'date',
        'caption': 'text',
        'likesCount': 'likes',
        'commentsCount': 'comments',
        'videoPlayCount': 'views',
        'type': 'type',
        'videoDuration': 'video_duration'
        })
    
    # ### TikTok
    
    # In[19]:
    
    
    # TikTok Parameters
    
    tt_max_results = max_results
    tt_past_days = past_days_posts
    
    
    # In[20]:
    
    if tt_profile_names:
        run_input = {
        
            "profiles": tt_profile_names,
            "scrapeLastNDays": tt_past_days,
            "resultsPerPage": tt_max_results
        
        }
        
        # Run the Actor and wait for it to finish
        run = apify.actor("clockworks/free-tiktok-scraper").call(run_input=run_input)
        
        i = 0
        data = {}
        for item in apify.dataset(run["defaultDatasetId"]).iterate_items():
            data[i] = item
            i = i +1
        
        videos = pd.DataFrame(data).transpose()
        videos = videos[videos['createTime'].notna()]
        
        videos = videos.drop(['musicMeta','mediaUrls'], axis=1)
        author_df = pd.json_normalize(videos['authorMeta'])
        author_df = author_df.rename(columns=lambda x: 'author_' + x)
        video_df = pd.json_normalize(videos['videoMeta'])
        video_df = video_df.rename(columns=lambda x: 'video_' + x)
        videos = pd.concat([videos.drop(columns=['authorMeta','videoMeta']), author_df,video_df], axis=1)
        videos = videos[videos['createTime'].notna()]
        videos = videos.rename(columns={"id": "video_id"})
        tt_posts = videos[['author_profileUrl', 'video_id', 'author_name', 'webVideoUrl', 'createTimeISO', 'text', 'diggCount', 'commentCount', 'shareCount', 'playCount', 'collectCount', 'video_duration', 'author_following', 'author_friends', 'author_fans', 'author_heart', 'author_video', 'author_digg']]
        tt_posts['type'] = 'video'
        
        tt_video_list = videos[['webVideoUrl']]['webVideoUrl'].to_list()
        
        tt_posts = tt_posts.rename(columns={
        'author_profileUrl': 'profile_url',
        'video_id': 'id',
        'author_name': 'user_name',
        'webVideoUrl': 'url',
        'createTimeISO': 'date',
        'text': 'text',
        'diggCount': 'likes',
        'commentCount': 'comments',
        'shareCount': 'shares',
        'playCount': 'views',
        'collectCount': 'bookmarks',
        'video_duration': 'video_duration'
        })
    # ## Analysis
    
    # ### Combine into a single dataframe  

    available_posts = []

    if 'fb_posts' in locals():
        available_posts.append(fb_posts)
    if 'inst_posts' in locals():
        available_posts.append(inst_posts)
    if 'tt_posts' in locals():
        available_posts.append(tt_posts)

    posts = pd.concat(available_posts, ignore_index=True)
    
    expected_columns = [
        'profile_url', 'id', 'user_name', 'url', 'date', 'text',
        'likes', 'comments', 'shares', 'views', 'type',
        'bookmarks', 'video_duration'
    ]

    # Keep only the columns that actually exist
    posts = posts[[col for col in expected_columns if col in posts.columns]]
    
    
    # In[260]:
    
    
    posts["platform"] = (
        posts["profile_url"]
          .str.extract(r"https?://(?:www\.)?([^/]+)")   # → 'facebook.com'
          [0]                                           # the capture group
          .str.split(".").str[0]                        # keep 'facebook'
          .str.lower()                                  # normalise
    )
    
    
    # In[261]:
    
    
    def secs(val):
        """
        Return total seconds or <NA> for anything empty/invalid.
        Handles plain-seconds (8, '18') or 'HH:MM:SS'.
        """
        # 1) missings ------------------------------------------------------------
        if val is None or (isinstance(val, float) and math.isnan(val)) or pd.isna(val):
            return pd.NA
    
        # 2) already numeric (int / numeric-string) ------------------------------
        try:
            return int(float(val))
        except (ValueError, TypeError):
            pass                        # fall through if it isn't plain seconds
    
        # 3) assume HH:MM:SS (or MM:SS) -----------------------------------------
        parts = list(map(int, str(val).split(":")))
        if len(parts) == 2:             # MM:SS
            parts = [0] + parts
        h, m, s = parts
        return h*3600 + m*60 + s
    
    # ── apply it ───────────────────────────────────────────────────────────────
    posts["video_duration"] = posts["video_duration"].apply(secs)
    
    
    # In[262]:
    
    
    posts['date'] = pd.to_datetime(posts['date'], format = 'mixed', utc = True)
    posts['type'] = posts['type'].str.title()
    posts['platform'] = posts['platform'].str.title()
    
    
    # In[263]:
    
    
    # Lookup Username 
    posts = posts.rename(columns={'profile_name':'user_name'})
    posts['user_name'] = posts['user_name'].str.lower()
    posts = posts.merge(lookup_df, on=['platform', 'user_name'], how='left')

    return posts

In [2]:
def generate_report (posts):

    import pandas as pd
    from apify_client import ApifyClient
    import json
    pd.options.display.max_columns = None
    from datetime import datetime, timedelta, date
    from pandas import json_normalize
    import numpy as np
    import math
    from sqlalchemy.exc import DatabaseError
    from openai import OpenAI, AsyncOpenAI
    import os
    from tqdm import tqdm
    tqdm.pandas()
    from typing import Dict, List
    import tiktoken
    import nest_asyncio, asyncio
    import dateparser
    import plotly
    import matplotlib.pyplot as plt
    import matplotlib
    import matplotlib.ticker as ticker
    from matplotlib.backends.backend_pdf import PdfPages
    from PIL import Image
    from docx import Document
    from docx.shared import Inches
    from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
    from docx.shared import Inches, Pt
    from docx.oxml.ns import qn
    from docx.oxml import OxmlElement
    from docx.enum.table import WD_ALIGN_VERTICAL
    from docx.shared import Pt, RGBColor
    from io import BytesIO
    from docx2pdf import convert
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    from IPython.display import FileLink, HTML, display
    from urllib.parse import quote
    from pathlib import Path
    import pathlib
    import base64
    
    posts = posts
    
    # ### Number of Posts
    
    # In[281]:
    
    
    number = posts.pivot_table(index='account_name', columns='platform', aggfunc='count', values='id', margins=True, margins_name='Total')
    
    
    # In[282]:
    
    
    # Drop the 'Total' row and column
    data = number.drop(index='Total', columns='Total')
    
    # Define custom colors in column order
    platform_colors = {
        'Facebook': '#1877F2',   # Facebook Blue
        'Instagram': '#E1306C',  # Instagram Pink
        'Tiktok': '#69C9D0'      # TikTok Aqua
    }
    colors = [platform_colors[col] for col in data.columns]
    
    # Plot with custom colors
    ax = data.plot(kind='bar', stacked=True, figsize=(8, 6), color=colors)
    
    # Beautify
    plt.title('Platform Usage by Brand - Last 2 Weeks')
    plt.xlabel('')
    plt.xticks(rotation=0, fontsize=10)
    plt.ylabel('Number of Posts')
    plt.legend(title='Platform', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    fig = ax.get_figure()
    fig.savefig('plot1.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    
    # ### Top Posts
    
    # In[283]:
    
    
    top_comments = posts.pivot_table(index='account_name', aggfunc='max', values=['comments']).reset_index()
    top_comments = top_comments.merge(posts[['comments','url']], how='left', on='comments')
    
    
    # In[284]:
    
    
    top_views = posts.pivot_table(index='account_name', aggfunc='max', values=['views']).reset_index()
    top_views = top_views.merge(posts[['views','url']], how='left', on='views')
    
    
    # In[285]:
    
    
    top_shares = posts.pivot_table(index='account_name', aggfunc='max', values=['shares']).reset_index()
    top_shares = top_shares.merge(posts[['shares','url']], how='left', on='shares')
    
    
    # In[286]:
    
    
    top = posts.pivot_table(index='account_name', aggfunc='max', values=['comments', 'views', 'shares']).reset_index()
    
    
    # In[287]:
    
    
    # Data setup
    top_data = top.set_index('account_name')
    left_data = top_data[['comments', 'shares']]
    right_data = top_data['views']
    
    fig, ax1 = plt.subplots(figsize=(8, 6))
    
    bar_width = 0.25
    index = range(len(top_data))
    
    # Left axis: comments and shares
    ax1.bar([i - bar_width/2 for i in index], left_data['comments'], width=bar_width, label='Comments', color='#66c2a5')
    ax1.bar([i + bar_width/2 for i in index], left_data['shares'], width=bar_width, label='Shares', color='#fc8d62')
    ax1.set_ylabel('Comments / Shares', fontsize=12)
    ax1.set_xticks(index)
    ax1.set_xticklabels(top_data.index, rotation=0, fontsize=10)
    ax1.legend(loc='upper left')
    
    # Right axis: views
    ax2 = ax1.twinx()
    ax2.bar([i + bar_width*1.5 for i in index], right_data, width=bar_width, label='Views', color='#8da0cb')
    ax2.set_ylabel('Views', fontsize=12, color='#8da0cb')
    ax2.tick_params(axis='y', labelcolor='#8da0cb')
    
    # Format y-ticks on right axis to millions
    ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x/1_000_000:.1f}M'))
    
    # Separate legend for views
    ax2.legend(loc='upper right')
    
    # Final touches
    plt.title("Top Posts' Engagement - Last 2 Weeks", fontsize=14)
    plt.tight_layout()
    plt.grid(axis='y', linestyle=':', alpha=0.6)
    
    fig.savefig('plot2.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    
    # ### Video Duration
    
    # In[288]:
    
    
    temp = posts[posts['video_duration']>0]
    duration = temp.pivot_table(index='account_name', values='video_duration', aggfunc=['mean', 'max', 'min', 'count']).reset_index()
    
    
    # In[289]:
    
    
    # Extract values
    x = duration['account_name']
    means = duration[('mean', 'video_duration')]
    mins = duration[('min', 'video_duration')]
    maxs = duration[('max', 'video_duration')]
    
    # Calculate asymmetric error bars
    lower_errors = means - mins
    upper_errors = maxs - means
    asymmetric_errors = [lower_errors, upper_errors]
    
    # Plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.bar(x, means, yerr=asymmetric_errors, capsize=10, color=['#66c2a5', '#fc8d62'])
    
    # Beautify
    ax.set_ylabel('Average Video Duration (s)', fontsize=12)
    ax.set_title('Video Duration Summary - Last 2 Weeks', fontsize=14)
    ax.set_xticks(range(len(x)))
    ax.set_xticklabels(x, rotation=0, fontsize=10)
    ax.grid(axis='y', linestyle=':', alpha=0.6)
    
    plt.tight_layout()
    fig.savefig('plot3.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    
    # ### Type of Posts
    
    # In[290]:
    
    
    posttype = posts.pivot_table(index='account_name', columns='type', aggfunc='count', values=['id'], margins=True, margins_name='Total').reset_index()
    
    
    # In[291]:
    
    
    # Flatten MultiIndex columns
    posttype.columns = ['_'.join(col).strip() if col[1] else col[0] for col in posttype.columns]
    
    # Now columns are like: 'account_name', 'id_Image', ..., 'id_Total'
    # Prepare data
    data = posttype[posttype['account_name'] != 'Total'].copy()
    
    # Drop id_Total if present
    if 'id_Total' in data.columns:
        data = data.drop(columns='id_Total')
    
    # Set account_name as index
    data = data.set_index('account_name').fillna(0)
    
    # Clean column names for display
    clean_labels = [col.replace('id_', '') for col in data.columns]
    colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3']
    
    # Function to skip small slice %s
    def autopct_format(pct):
        return f'{pct:.1f}%' if pct > 5 else ''
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    fig.suptitle('Type of Posts – Last 2 Weeks', fontsize=16, y=1.05)
    
    for ax, (account, row) in zip(axes, data.iterrows()):
        ax.pie(
            row,
            labels=clean_labels,
            autopct=autopct_format,
            startangle=90,
            colors=colors,
            labeldistance=1.1,
            wedgeprops={'linewidth': 1, 'edgecolor': 'white'}
        )
        ax.set_title(account, fontsize=14)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    fig.savefig('plot4.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    print("Done Until Report")
    # ## Create Report
    
    # In[292]:
    
    
    def df_to_docx_table(doc, df, title=None):
        if title:
            heading = doc.add_paragraph(title)
            heading.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            heading.style = 'Heading 2'
    
        table = doc.add_table(rows=1, cols=len(df.columns))
        table.style = 'Table Grid'
    
        # Header row
        hdr_cells = table.rows[0].cells
        for i, col in enumerate(df.columns):
            hdr_cells[i].text = str(col)
    
        # Data rows
        for index, row in df.iterrows():
            row_cells = table.add_row().cells
            for i, val in enumerate(row):
                row_cells[i].text = str(val)
    
        doc.add_paragraph()  # Add spacing between tables
    
    
    # In[293]:
    
    
    # Extract the account names (excluding 'Total')
    accounts = posts['account_name'].unique()
    report_title = f"Social Media - Last Two Weeks - Basic Report: {accounts[0]} vs {accounts[1]}"
    
    # In[295]:
    
    
    # === CREATE WORD DOCUMENT ===
    doc = Document()
    # Add title as custom paragraph (not using style='Title')
    title_paragraph = doc.add_paragraph()
    title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
    run = title_paragraph.add_run(report_title)
    run.font.name = 'Orbitron'
    run.font.size = Pt(28)
    run.font.bold = True
    run.font.color.rgb = RGBColor(0, 0, 0)  # Black
    
    for _ in range(6):
        doc.add_paragraph()
    img_path = Path.cwd() / "Analytics.png"
    doc.add_picture(str(img_path), width=Inches(6))
    doc.add_page_break()
    
    
    plot_files = [
        ('plot1.png', 'Platform Usage'),
        ('plot2.png', 'Engagement Metrics'),
        ('plot3.png', 'Video Duration Summary Statistics'),
        ('plot4.png', 'Type of Posts')
    ]

    print(plot_files)
    
    for i, (plot_file, heading_text) in enumerate(plot_files):
        heading_para = doc.add_paragraph()
        heading_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        run = heading_para.add_run(heading_text)
        run.font.name = 'Orbitron'
        run.font.size = Pt(19)
        run.font.bold = True
        run.font.color.rgb = RGBColor(187, 20, 26)  # #BB141A
    
        # Add a full-height table for vertical centering of image only
        table = doc.add_table(rows=1, cols=1)
        table.allow_autofit = False
        cell = table.cell(0, 0)
        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    
        # Set full page height minus a bit for the heading (approx 8 inches)
        row = table.rows[0]
        row.height = Inches(8)
    
        # Ensure <w:tblPr> exists
        tbl = table._tbl
        tbl_pr = tbl.tblPr
        if tbl_pr is None:
            tbl_pr = OxmlElement('w:tblPr')
            tbl.append(tbl_pr)
    
        tbl_w = OxmlElement('w:tblW')
        tbl_w.set(qn('w:w'), '0')
        tbl_w.set(qn('w:type'), 'auto')
        tbl_pr.append(tbl_w)
    
        # Add image into the cell
        paragraph = cell.paragraphs[0]
        run = paragraph.add_run()
        run.add_picture(plot_file, width=Inches(6))
        paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    
        # Add a page break unless it's the last
        if i != len(plot_files) - 1:
            doc.add_page_break()

    print("Done with Graphs in Report")
    
    # Add all dataframes on one page
    doc.add_page_break()
    heading_para = doc.add_paragraph()
    heading_para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run = heading_para.add_run('Underlying Data Tables')
    run.font.name = 'Orbitron'
    run.font.size = Pt(19)
    run.font.bold = True
    run.font.color.rgb = RGBColor(187, 20, 26)  # #BB141A

    number = number.reset_index()
    
    df_to_docx_table(doc, number, 'Platform Usage')
    df_to_docx_table(doc, top_views, 'Most Viewed Posts')
    df_to_docx_table(doc, top_comments, 'Most Commented Posts')
    df_to_docx_table(doc, top_shares, 'Most Shared Posts')
    df_to_docx_table(doc, top, 'Top Posts')
    df_to_docx_table(doc, duration.droplevel(0, axis=1), 'Video Duration Summary')  # if multiindex
    df_to_docx_table(doc, posttype, 'Post Type Count')
    
    # Save Word file
    file_name = f"Social Media - Last Two Weeks - Basic Report - {accounts[0]} vs {accounts[1]}.docx"
    doc.save(file_name)
    with open(file_name, "rb") as fp:
        b64 = base64.b64encode(fp.read()).decode()
    
    mime = "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
    html = (f'<a download="{file_name}" '
            f'href="data:{mime};base64,{b64}">'
            '📄 Download the Word report</a>')
    display(HTML(html))
    #convert(file_name) This only works if you are running the script from a Windows PC with Word installed. Can use a cloud conversion service, but unnecessary at this stage.
    
    
    # In[296]:
    
    
    for file_tuple in plot_files:
        filename = file_tuple[0]  # extract just the filename
        if os.path.exists(filename):
            os.remove(filename)




In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------- widgets ----------
btn_generate = widgets.Button(
    description="Generate report",
    button_style="success",
    icon="check"
)
out_log    = widgets.Output(layout={"border": "1px solid #888",
                                    "height": "200px",
                                    "overflow": "auto"})
out_report = widgets.Output(layout={"border": "1px solid #888",
                                    "padding": "6px"})

# ---------- click handler ----------
def _on_click(b):
    btn_generate.disabled = True
    with out_log:
        clear_output()
        print("Running fetch_data() …")
    try:
        posts = fetch_data()                           # ①

        with out_log:
            print("✅  Data fetched, building report …")

        with out_report:                              # ② capture all displays
            clear_output()
            generate_report(posts)                    # build DOCX & show link

        with out_log:
            print("🎉  All done")
    except Exception as e:
        with out_log:
            print("🚫  Error:", e)
            raise
    finally:
        btn_generate.disabled = False

btn_generate.on_click(_on_click)

display(btn_generate, out_log, out_report)


Button(button_style='success', description='Generate\u202freport', icon='check', style=ButtonStyle())

Output(layout=Layout(border='1px solid #888', height='200px', overflow='auto'))

Output(layout=Layout(border='1px solid #888', padding='6px'))